In [18]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
import category_encoders as ce
from sklearn.impute import SimpleImputer
from sklearn.pipeline import make_pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.dummy import DummyClassifier
from sklearn.metrics import roc_auc_score
from sklearn.metrics import accuracy_score

In [8]:
people = pd.read_csv('../DS-Unit-2-4-Build-Week/people.csv')

C:\Users\Westopher\Coding\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3058: DtypeWarning: Columns (11,12,19,20,21,22,23,24,25,26,28) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [9]:
people.describe().T

,count,mean,std,min,25%,50%,75%,max
VEHICLE_ID,768713.0,372184.236701,214710.997314,2.0,187073.00,371645.00,557891.00,744984.00
SEAT_NO,153425.0,4.208121,2.169906,1.0,3.00,3.00,6.00,12.00
AGE,564067.0,38.063264,17.099626,-49.0,26.00,36.00,50.00,110.00
BAC_RESULT VALUE,952.0,0.172027,0.098773,0.0,0.13,0.17,0.22,0.99


In [10]:
people.describe(include = object).T

,count,unique,top,freq
PERSON_ID,783816,783816,O409908,1
PERSON_TYPE,783816,6,DRIVER,615209
RD_NO,783816,357877,JB187770,61
CRASH_DATE,783816,229497,11/10/2017 10:30:00 AM,64
CITY,585027,8453,CHICAGO,410601
STATE,589541,52,IL,558262
ZIPCODE,536370,9395,60629,19219
SEX,773346,4,M,410832
DRIVERS_LICENSE_STATE,473191,171,IL,436438
DRIVERS_LICENSE_CLASS,417338,215,D,363711


In [22]:
people['INJURY_CLASSIFICATION'].isnull().sum()

0

In [21]:
people = people.dropna(subset=['INJURY_CLASSIFICATION'])

In [23]:
people['INJURY_CLASSIFICATION'].value_counts(normalize = True)

NO INDICATION OF INJURY     0.925683
NONINCAPACITATING INJURY    0.040472
REPORTED, NOT EVIDENT       0.025410
INCAPACITATING INJURY       0.008032
FATAL                       0.000403
Name: INJURY_CLASSIFICATION, dtype: float64

In [24]:
X_train, X_test, y_train, y_test = train_test_split(people.drop(['INJURY_CLASSIFICATION'], axis = 1), 
                                                    people['INJURY_CLASSIFICATION'], test_size=0.20)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((626784, 28), (156696, 28), (626784,), (156696,))

In [25]:
pipeline = make_pipeline(
    ce.OrdinalEncoder(),
    SimpleImputer(),
    RandomForestClassifier(n_estimators = 100, n_jobs = -1)    
)

pipeline.fit(X_train, y_train)

# y_pred = pipeline.predict(X_valid)
# accuracy_score(y_valid, y_pred)

cross_val_score(pipeline, X_train, y_train, scoring='accuracy', cv=5)

array([0.9358642 , 0.94845122, 0.94816404, 0.94833157, 0.93487244])